# 📉 PyKwant Tutorial: Market Risk (VaR & ES)

This notebook introduces `pykwant.market_risk`, a module dedicated to probabilistic risk measurement.

While `risk.py` measures sensitivity (Greeks), `market_risk.py` answers the question: **"How much money could I lose?"**

We cover:
1. **Value at Risk (VaR)**: The maximum loss at a specific confidence level (e.g., 99%).
2. **Expected Shortfall (ES)**: The average loss *if* the VaR threshold is breached (Tail Risk).
3. **Approaches**: Comparison between Parametric (Normal) and Historical methodologies.

## 1. Setup and Data Simulation

We simulate a portfolio with a value of **$1,000,000** and generate 1 year (252 days) of synthetic historical returns.

To make it interesting, we will simulate returns that are mostly normal but include a "Market Crash" event (Fat Tail) to demonstrate where Historical simulation shines.

In [1]:
import math
import random

from pykwant import analytics, market_risk

# Setup
portfolio_value = 1_000_000.0
random.seed(42)

# 1. Generate Synthetic Returns (Normal Distribution)
# Mean 0, Vol 20% Annual
sigma_daily = 0.20 / math.sqrt(252)
returns = [random.gauss(0, sigma_daily) for _ in range(250)]

# 2. Inject a "Black Swan" event (e.g., -5% drop)
# This violates the Normal distribution assumption
returns.extend([-0.05, -0.04])

print(f"Simulated {len(returns)} days of returns.")
print(f"Worst Return: {min(returns):.2%}")

# Helper for display
def print_risk(label, value):
    print(f"{label:<25}: ${value:,.2f}")

Simulated 252 days of returns.
Worst Return: -5.00%


## 2. Parametric Value at Risk (Normal VaR)

The Parametric approach assumes returns follow a Gaussian distribution. It is fast and requires only the volatility.

$$ VaR = V \times \sigma \times z_{\alpha} $$

We calculate the 95% and 99% VaR for a 1-day horizon.

In [2]:
# 1. Calculate Volatility from data
vol_annual = analytics.annualized_volatility(returns)
print(f"Annualized Volatility: {vol_annual:.2%}")

# 2. Calculate Parametric VaR
var_95_param = market_risk.parametric_var(portfolio_value, vol_annual, confidence_level=0.95)
var_99_param = market_risk.parametric_var(portfolio_value, vol_annual, confidence_level=0.99)

print("\n--- Parametric VaR (Normal Assumption) ---")
print_risk("VaR 95% (1 Day)", var_95_param)
print_risk("VaR 99% (1 Day)", var_99_param)

Annualized Volatility: 20.49%

--- Parametric VaR (Normal Assumption) ---
VaR 95% (1 Day)          : $21,230.77
VaR 99% (1 Day)          : $30,027.08


## 3. Historical Value at Risk

The Historical approach does not make assumptions about the distribution. It simply looks at what happened in the past (percentiles).

Because we injected a **-5% crash** in our data, the Historical VaR at 99% should be significantly higher than the Parametric estimate (which "ignores" fat tails).

In [3]:
var_95_hist = market_risk.historical_var(portfolio_value, returns, confidence_level=0.95)
var_99_hist = market_risk.historical_var(portfolio_value, returns, confidence_level=0.99)

print("--- Historical VaR (Actual Data) ---")
print_risk("VaR 95% (1 Day)", var_95_hist)
print_risk("VaR 99% (1 Day)", var_99_hist)

diff = var_99_hist - var_99_param
print(f"\nDifference at 99%: ${diff:,.2f} (Impact of Fat Tails)")

--- Historical VaR (Actual Data) ---
VaR 95% (1 Day)          : $19,390.04
VaR 99% (1 Day)          : $31,439.94

Difference at 99%: $1,412.86 (Impact of Fat Tails)


## 4. Expected Shortfall (CVaR)

VaR tells us the threshold, but not *how bad* things can get if that threshold is crossed.
**Expected Shortfall (ES)** averages the losses in the tail (the worst 5% or 1%).

This is often considered a coherent risk measure, unlike VaR.

In [4]:
es_95_param = market_risk.parametric_expected_shortfall(
    portfolio_value, vol_annual, confidence_level=0.95
)
es_95_hist = market_risk.historical_expected_shortfall(
    portfolio_value, returns, confidence_level=0.95
)

print("--- Expected Shortfall (95% Confidence) ---")
print_risk("Parametric ES", es_95_param)
print_risk("Historical ES", es_95_hist)

# Historical ES is heavily impacted by the single -5% day
# because it pulls the tail average down significantly.

--- Expected Shortfall (95% Confidence) ---
Parametric ES            : $26,624.24
Historical ES            : $27,907.27


## 5. Summary Report

A typical risk report compares these metrics side-by-side.

In [5]:
print(f"{'Metric':<20} | {'Parametric':>15} | {'Historical':>15}")
print("-" * 56)
print(f"{'VaR 95%':<20} | {var_95_param:>15,.2f} | {var_95_hist:>15,.2f}")
print(f"{'VaR 99%':<20} | {var_99_param:>15,.2f} | {var_99_hist:>15,.2f}")
print(f"{'ES 95%':<20}  | {es_95_param:>15,.2f} | {es_95_hist:>15,.2f}")

Metric               |      Parametric |      Historical
--------------------------------------------------------
VaR 95%              |       21,230.77 |       19,390.04
VaR 99%              |       30,027.08 |       31,439.94
ES 95%                |       26,624.24 |       27,907.27
